# Finetuning BLOOM for NER: Compile Corpus

***

## Imports

In [1]:
from transformers import (BloomTokenizerFast,
                          BloomForTokenClassification,
                          DataCollatorForTokenClassification, 
                          AutoModelForTokenClassification, 
                          TrainingArguments, Trainer)
from datasets import load_dataset, concatenate_datasets, DatasetDict
import pickle
import torch
import os

/home/julian/Documents/Workspace_Python/bloom-ner-multilingual/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Tokenizer

The list of available Models can be found here: https://huggingface.co/docs/transformers/model_doc/bloom

In [2]:
model_name = "bloom-560m"
tokenizer = BloomTokenizerFast.from_pretrained(f"bigscience/{model_name}", add_prefix_space=True)
#model = BloomForTokenClassification.from_pretrained(f"bigscience/{model_name}")

## Download Dataset for Finetuning

See:
* Dataset on Huggingface: https://huggingface.co/datasets/wikiann
* Load Datasets: https://huggingface.co/docs/datasets/v2.4.0/en/package_reference/loading_methods

In [4]:
# Specify list of languages
languages = ["en","de", "fr", "es", "zh"]
#languages = ["en"]
dataset_name = "wikiann"

# Downloa first language
dataset_train = load_dataset(dataset_name, languages[0],  split="train")
dataset_valid = load_dataset(dataset_name, languages[0],  split="validation")
dataset_test =  load_dataset(dataset_name, languages[0],  split="test")
languages.pop(0)

# Merge with additional languages
for language in languages:
    
    print(f"Download Dataset for Language {language}")
    
    # Combine train splits
    dataset_train_new = load_dataset(dataset_name, language,  split="train")
    dataset_train = concatenate_datasets([dataset_train, dataset_train_new])

    # Combine validation splits
    dataset_valid_new = load_dataset(dataset_name, language,  split="validation")
    dataset_valid = concatenate_datasets([dataset_valid, dataset_valid_new])
    
    # Combine test splits
    dataset_test_new = load_dataset(dataset_name, language,  split="test")
    dataset_test = concatenate_datasets([dataset_test, dataset_test_new])

Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/en/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


Download Dataset for Language de


Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/de/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/de/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/de/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


Download Dataset for Language fr


Dataset wikiann downloaded and prepared to /home/julian/.cache/huggingface/datasets/wikiann/fr/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


...

Dataset wikiann downloaded and prepared to /home/julian/.cache/huggingface/datasets/wikiann/es/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/es/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/es/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


Download Dataset for Language zh


Dataset wikiann downloaded and prepared to /home/julian/.cache/huggingface/datasets/wikiann/zh/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e. Subsequent calls will reuse this data.


Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/zh/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)
Reusing dataset wikiann (/home/julian/.cache/huggingface/datasets/wikiann/zh/1.1.0/4bfd4fe4468ab78bb6e096968f61fab7a888f44f9d3371c2f3fea7e74a5a354e)


In [5]:
dataset = DatasetDict({
    "train":dataset_train,
    "test":dataset_test, 
    "validation":dataset_valid
    })

**Limit Dataset Size for Testing:**

In [7]:
## Sample a subset of datapoints
#num_samples = 1000
#sample_ids = list(range(0,num_samples))
#
## Reduce the size of the dataset
#dataset_train = dataset_train.select(sample_ids)
#dataset_valid = dataset_valid.select(sample_ids)
#dataset_test = dataset_test.select(sample_ids)
#
#print("Training Examples:", len(dataset_train))

**Save combined Dataset:**

In [6]:
data_path = "./data/dataset_multilingual.pkl"
with open(data_path, 'wb') as pickle_file:
    pickle.dump(obj = dataset, file=pickle_file)

### About the Dataset:

**Splits:**

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 100000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'spans'],
        num_rows: 50000
    })
})

**Training Examples:**

In [8]:
print("Dataset Object Type:", type(dataset["train"]))
print("Training Examples:", len(dataset["train"]))

Dataset Object Type: <class 'datasets.arrow_dataset.Dataset'>
Training Examples: 100000


**Sample Structure:**

In [9]:
dataset["train"][95]

{'tokens': ['Bruce', 'Beresford', '(', 'Non-Jew', ')'],
 'ner_tags': [1, 2, 0, 0, 0],
 'langs': ['en', 'en', 'en', 'en', 'en'],
 'spans': ['PER: Bruce Beresford']}

**Class Labels:**

In [10]:
label_list = dataset["train"].features[f"ner_tags"].feature.names
print(label_list)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']
